# RAG Pipeline Exercise

In this exercise you will build and **compare two simple Retrieval-Augmented Generation (RAG) pipelines**.

You will work with a small collection of PDF documents (e.g. medical guidelines) and:

1. Load and chunk the PDF documents.
2. Create a vector index using **embedding model A** (local `BAAI/bge-m3`).
3. Create a second index using **embedding model B** (e.g. OpenAI or Gemini embeddings).
4. Implement a simple **retriever** and an **answering function** that calls an LLM with retrieved context.
5. Automatically **generate questions** from the documents and use them to **compare two RAG configurations**.

Cells marked with `# TODO` are **for students to implement**.
Everything else is provided scaffolding.

## 0. Setup & Imports

In [1]:
# TODO (easy): skim the imports and make sure you understand what each library is used for.

from dotenv import load_dotenv
import os
import glob
from PyPDF2 import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import faiss
from sentence_transformers import SentenceTransformer
import pickle
import random
import numpy as np
import pandas as pd

# LLM / API clients (we will mainly use OpenAI here; Gemini can be added as a bonus)
from openai import OpenAI

In [2]:
# Load API keys from .env (you need to create this file once and add your keys)
load_dotenv()

deepinfra_key = os.getenv("DEEPINFRA_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

# For this exercise we mainly use OpenAI for both embeddings (RAG B) and chat completions.
assert openai_api_key is not None, "Please set OPENAI_API_KEY in your .env file."
openai_client = OpenAI(api_key=openai_api_key)


In [3]:
# Make pandas show the full table and full cell content
pd.set_option("display.max_rows", None)       # show all rows
pd.set_option("display.max_columns", None)    # show all columns
pd.set_option("display.max_colwidth", None)   # don't truncate cell text

## 1. Load PDF documents

We assume there is a `data/` folder containing one or more PDF files.

**Task:** implement `load_pdfs(glob_path)` so that it:
- Iterates over all PDF files matching `glob_path`
- Reads them with `PdfReader`
- Concatenates the text of all pages into **one long string**.

In [4]:
def load_pdfs(glob_path: str = "data/*.pdf") -> str:
    """Load all PDFs matching the pattern and return their combined text.

    TODO:
    - Use `glob.glob(glob_path)` to iterate over file paths
    - For each file, open it in binary mode and create a `PdfReader`
    - Loop over `reader.pages` and extract text with the extract_text() function
    - Concatenate everything into a single string `text`
    - Be robust: skip pages where `extract_text()` returns None
    """
    # YOUR CODE HERE
    text = ""
    for pdf_path in glob.glob(glob_path):
        with open(pdf_path, "rb") as f:
            reader = PdfReader(f)
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += " " + page_text
    return text



In [5]:
# Run once and inspect
raw_text = load_pdfs("data/*.pdf")
print("Number of characters:", len(raw_text))
print("Preview:", raw_text[:500])

Number of characters: 230708
Preview:  Asthma: diagnosis, 
moni toring and chr onic 
asthma manag emen t (BTS, 
NICE, SI GN) 
NICE guideline 
Published: 27 No vember 202 4 
www .nice.or g.uk/guidance/ng2 45 
© NICE 202 4. All right s reserved. Subject t o Notice of right s (https://www .nice.or g.uk/t erms-and-
conditions#notice-of -right s). Your r esponsi bility 
The r ecommendations in t his guideline r epresent t he view of NICE, arriv ed at aft er car eful 
consideration of t he evidence a vailable. When e xercising t heir judg


## 2. Chunk the text

We will split the long text into overlapping chunks.

Later you can **experiment** with different `chunk_size` and `chunk_overlap` to see how it affects retrieval.

**Task:** start with the given parameters, run once, then try at least one alternative configuration and note the effects.

In [6]:
# Base configuration (RAG A)
chunk_size_a = 2000
chunk_overlap_a = 200

splitter_a = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size_a,
    chunk_overlap=chunk_overlap_a
)

chunks_a = splitter_a.split_text(raw_text)
print(f"RAG A: {len(chunks_a)} chunks produced, first chunk length = {len(chunks_a[0])}")

# TODO (mini-experiment): change chunk_size / chunk_overlap for RAG B and compare
chunk_size_b = 1000   # e.g. smaller chunks
chunk_overlap_b = 100

splitter_b = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size_b,
    chunk_overlap=chunk_overlap_b
)

chunks_b = splitter_b.split_text(raw_text)
print(f"RAG B: {len(chunks_b)} chunks produced, first chunk length = {len(chunks_b[0])}")

RAG A: 130 chunks produced, first chunk length = 1995
RAG B: 260 chunks produced, first chunk length = 979


## 3. Create embeddings and a FAISS index

We start with **Embedding model A: `BAAI/bge-small-en`** using `sentence-transformers`.

Then, as an optional extension, you can build **Embedding model B** using OpenAI or Gemini and compare.

To keep the exercise manageable, the base version only **requires** BGE.

In [7]:
# Embedding model A (local)
model_name_a = "BAAI/bge-small-en"
embedder_a = SentenceTransformer(model_name_a)

# Compute embeddings for all chunks of configuration A
embeddings_a = embedder_a.encode(chunks_a, convert_to_numpy=True)

dimensions_a = embeddings_a.shape[1]
print("Embedding dimensionality (A):", dimensions_a)

index_a = faiss.IndexFlatL2(dimensions_a)
index_a.add(embeddings_a)
print("FAISS index (A) size:", index_a.ntotal)

# Persist index/chunks if you like (optional)
os.makedirs("faiss", exist_ok=True)
faiss.write_index(index_a, "faiss/faiss_index_a.index")
with open("faiss/chunks_a.pkl", "wb") as f:
    pickle.dump(chunks_a, f)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\Phil\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Phil\.cache\huggingface\hub\models--BAAI--bge-small-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding dimensionality (A): 384
FAISS index (A) size: 130


In [8]:
# Embedding model B using OpenAI embeddings.

# TODO:
# - Use `openai_client.embeddings.create(...)` to compute embeddings for `chunks_b`
# - Create a second FAISS index `index_b`
# - Make sure to check the dimensionality from the first embedding vector

# Example sketch (not complete, adapt & run if you have API access):
# Initialize OpenAI client
openai_client = OpenAI(api_key=openai_api_key)
response = openai_client.embeddings.create(
     model="text-embedding-3-small",
    input=chunks_b
)
embeddings_b = np.array([item.embedding for item in response.data])
dim_b = embeddings_b.shape[1]
index_b = faiss.IndexFlatL2(dim_b)
index_b.add(embeddings_b)
print("FAISS index (B) size:", index_b.ntotal)

FAISS index (B) size: 260


## 4. Implement a simple retriever

We now implement a generic retrieval function that:
1. Embeds the query.
2. Searches the FAISS index.
3. Returns the corresponding text chunks.

We implement it for configuration A. If you built configuration B, you can reuse the same function.

In [9]:
def retrieve_texts(query: str, k: int, index, chunks, embedder) -> list:
    """Return the top-k most similar chunks for a query.

    TODO (students):
    - Encode the query with `embedder.encode(...)`
    - Call `index.search(query_embedding, k)`
    - Use the returned indices to select the chunks
    - Return a list of strings (chunks)
    """
    # YOUR CODE HERE
    query_emb = embedder.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_emb, k)
    retrieved = [chunks[i] for i in indices[0]]
    return retrieved

# Quick sanity check
test_query = "What is the most important factor in diagnosing asthma?"
retrieved_text = retrieve_texts(test_query, k=3, index=index_a, chunks=chunks_a, embedder=embedder_a)
print("Number of retrieved chunks:", len(retrieved_text))
print("Preview of first chunk:", retrieved_text[0][:400])

Number of retrieved chunks: 3
Preview of first chunk: and signs of ot her causes of r espirat ory sympt oms but be awar e that e ven if 
examination r esult s are normal, t he person ma y still ha ve ast hma. [NICE 2017] 
Initial tr eatmen t and obje ctive tests f or acu te sym ptoms a t 
presen tation 
1.1.5 Treat people immediat ely if t hey are acut ely unw ell or highly sympt omatic at 
presentation, and per form objectiv e tests that ma y help s


In [10]:
def openai_embed_query(query: str) -> np.ndarray:
    """
    Compute an OpenAI embedding for a single query string and
    return it as a NumPy array of shape (1, dim).
    """
    resp = openai_client.embeddings.create(
        model="text-embedding-3-small",
        input=[query]  # list with one string
    )
    vec = np.array(resp.data[0].embedding, dtype="float32")
    return vec.reshape(1, -1)

In [11]:
def retrieve_texts_b(query: str, k: int, index, chunks) -> list:
    """
    Retrieve top-k chunks using OpenAI embeddings for the query.
    """
    query_emb = openai_embed_query(query)  # shape (1, dim)
    distances, indices = index.search(query_emb, k)
    retrieved = [chunks[i] for i in indices[0]]
    return retrieved

## 5. Implement `answer_query` using an LLM

Now we build the actual RAG call:

1. Use `retrieve_texts` to get top-`k` chunks.
2. Concatenate them into a context string.
3. Build a prompt that:
   - shows the context
   - asks the model to answer the user question based **only** on this context.
4. Call the OpenAI chat completion API.

This is the **core RAG function**.

In [12]:
def answer_query_a(query: str, k: int, index, chunks, embedder, client: OpenAI) -> str:
    """RAG-style answer: retrieve context and ask an LLM.

    TODO (students):
    - Use `retrieve_texts` to get `k` relevant chunks.
    - Join them into a single context string.
    - Build a chat prompt that instructs the model to answer *only* using the context.
    - Call `client.chat.completions.create(...)` with model `"gpt-4o-mini"` (or similar).
    - Return the model's answer text.
    """
    retrieved_chunks = retrieve_texts(query, k, index, chunks, embedder)
    context = "\n\n---\n\n".join(retrieved_chunks)

    system_prompt = (
        "You are a helpful assistant answering questions based ONLY on the provided context. "
        "If the answer is not in the context, say that you do not know."
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {query}"}
    ]

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )

    return completion.choices[0].message.content.strip()

# Quick manual test
answer = answer_query_a(test_query, k=3, index=index_a, chunks=chunks_a, embedder=embedder_a, client=openai_client)
print("RAG answer:", answer)

RAG answer: The context does not specify a single most important factor in diagnosing asthma. It mentions several objective tests such as eosinophil count, fractional exhaled nitric oxide (FeNO), spirometry, and peak expiratory flow (PEF) which help support a diagnosis of asthma. Therefore, I do not know the most important factor in diagnosing asthma based on the provided context.


### Answer Function with the other embedding type

In [13]:
def answer_query_b(query: str, k: int, index, chunks, client: OpenAI) -> str:
    """
    RAG-style answer for configuration B (OpenAI embeddings + chunking B).
    """
    retrieved_chunks = retrieve_texts_b(query, k, index, chunks)
    context = "\n\n---\n\n".join(retrieved_chunks)

    system_prompt = (
        "You are a helpful assistant answering questions based ONLY on the provided context. "
        "If the answer is not in the context, say that you do not know."
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {query}"},
    ]

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )

    return completion.choices[0].message.content.strip()

## 6. Generate questions from random chunks (automatic evaluation set)

To compare two RAG configurations, we need **questions**.

We will:
- randomly sample a few chunks from the corpus,
- ask an LLM to generate a **good question** whose answer is contained in the chunk.

Then we can use these question–chunk pairs as a small evaluation set.

We provide most of the implementation. Your job is mainly to:
- inspect the code,
- understand the prompt,
- maybe tweak the number of chunks or retries.

In [14]:
def generate_questions_for_random_chunks(chunks, num_chunks: int = 5, max_retries: int = 2):
    selected_chunks = random.sample(chunks, num_chunks)
    qa_pairs = []

    for chunk in selected_chunks:
        prompt = prompt = (
            "Based on the following text, generate an insightful question that covers its key content:\n\n"
            "Text:\n" + chunk + "\n\n"
            "Question:"
        )

        question = None
        for attempt in range(max_retries):
            try:
                completion = openai_client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=[{"role": "user", "content": prompt}]
                )
                question = completion.choices[0].message.content.strip()
                if question:
                    break
            except Exception as e:
                print("Error while generating question, retrying...", e)

        if question is None:
            question = "Error: could not generate question."

        qa_pairs.append((chunk, question))

    return qa_pairs

questions = generate_questions_for_random_chunks(chunks_a, num_chunks=5, max_retries=2)
for i, (chunk, q) in enumerate(questions, 1):
    print(f"Q{i}: {q}\n  From chunk preview: {chunk[:120]}...\n")

Q1: What are the recommended clinic blood pressure targets for adults aged 80 and over with chronic kidney disease and varying levels of albumin to creatinine ratios, according to NICE's guidelines?
  From chunk preview: more or 
• chronic kidne y disease plus albumin 
to creatinine ratio of 7 0 mg/mmol or 
more Below 
130/80 NICE's guidel...

Q2: What challenges do healthcare providers face in diagnosing asthma in children, and what alternative methods does the committee suggest when standard diagnostic tests are unavailable or delayed?
  From chunk preview: conditions#notice-of -right s).Page 45 of
64 unlik ely and can be ruled out wit hout r esor ting t o bronchial challenge...

Q3: What are the key research questions being addressed regarding the effectiveness of relaxation therapies and appropriate blood pressure targets for managing hypertension in adults with specific conditions, as outlined in the NICE recommendations?
  From chunk preview: step 1 treatment . 
5 Relaxa tion the

## 7. Compare two RAG configurations

Now we can:
- Use the generated questions,
- Answer them with RAG configuration A (BGE + chunking A),
- (Optional) Answer them with RAG configuration B (e.g. different chunking and/or different embeddings),
- Compare the answers qualitatively.

To keep the exercise manageable, we start with config A only.
If you implemented config B, reuse `answer_query` with `index_b`, `chunks_b`, and your second embedder.

In [15]:
def answer_generated_questions_a(question_tuples, k, index, chunks, embedder, client):
    results = []
    for chunk, question in question_tuples:
        answer = answer_query_a(question, k, index, chunks, embedder, client)
        results.append({
            "chunk": chunk,
            "question": question,
            "answer": answer
        })
    return results

results_a = answer_generated_questions_a(
    questions,
    k=5,
    index=index_a,
    chunks=chunks_a,
    embedder=embedder_a,
    client=openai_client,
)

for item in results_a:
    print("Question:", item["question"])
    print("Answer A:", item["answer"])
    print("Source chunk preview:", item["chunk"][:150], "...")
    print("-" * 60)

Question: What are the recommended clinic blood pressure targets for adults aged 80 and over with chronic kidney disease and varying levels of albumin to creatinine ratios, according to NICE's guidelines?
Answer A: According to NICE's guidelines, the recommended clinic blood pressure targets for adults aged 80 and over with chronic kidney disease are as follows:

- For chronic kidney disease plus albumin to creatinine ratio of 70 mg/mmol or more: Below 130/80 mmHg.
- For chronic kidney disease plus albumin to creatinine ratio less than 70 mg/mmol: Below 140/90 mmHg.
Source chunk preview: more or 
• chronic kidne y disease plus albumin 
to creatinine ratio of 7 0 mg/mmol or 
more Below 
130/80 NICE's guideline on type 1 
diabet es in ad ...
------------------------------------------------------------
Question: What challenges do healthcare providers face in diagnosing asthma in children, and what alternative methods does the committee suggest when standard diagnostic tests are unavailab

### Extension: add RAG B and create a comparison table

If you implemented a second configuration (e.g. different chunking + OpenAI embeddings):

1. Build `index_b` using OpenAI embeddings and `chunks_b`.
2. Implement `openai_embed_query`, `retrieve_texts_b`, and `answer_query_b`.
3. Run `results_b = answer_generated_questions_b(questions, k=5, index=index_b, chunks=chunks_b, client=openai_client)`.
4. For each question, compare:
   - Which answer is more complete / specific?
   - Which one is better grounded in the source chunk?
5. Summarise your findings in a short **markdown cell** or a small table.

---

This concludes the core RAG exercise.


In [16]:
def answer_generated_questions_b(question_tuples, k, index, chunks, client):
    """
    Use RAG B to answer a list of (chunk, question) pairs.
    """
    results = []
    for chunk, question in question_tuples:
        answer = answer_query_b(question, k, index, chunks, client)
        results.append({
            "chunk": chunk,
            "question": question,
            "answer": answer,
        })
    return results


In [17]:
results_b = answer_generated_questions_b(
    questions,
    k=5,
    index=index_b,
    chunks=chunks_b,
    client=openai_client,
)

In [18]:
rows = []

for qa_a, qa_b in zip(results_a, results_b):
    rows.append({
        "Question": qa_a["question"],
        "Answer A (BGE + config A)": qa_a["answer"],
        "Answer B (OpenAI + config B)": qa_b["answer"],
        "Source chunk (A) preview": qa_a["chunk"][:200] + "..."
    })

df_comparison = pd.DataFrame(rows)
display(df_comparison)

,Question,Answer A (BGE + config A),Answer B (OpenAI + config B),Source chunk (A) preview
0,"What are the recommended clinic blood pressure targets for adults aged 80 and over with chronic kidney disease and varying levels of albumin to creatinine ratios, according to NICE's guidelines?","According to NICE's guidelines, the recommended clinic blood pressure targets for adults aged 80 and over with chronic kidney disease are as follows:\n\n- For chronic kidney disease plus albumin to creatinine ratio of 70 mg/mmol or more: Below 130/80 mmHg.\n- For chronic kidney disease plus albumin to creatinine ratio less than 70 mg/mmol: Below 140/90 mmHg.","For adults aged 80 and over with chronic kidney disease, the recommended clinic blood pressure targets according to NICE's guidelines are as follows:\n\n- If the albumin to creatinine ratio is less than 70 mg/mmol: Below 140/90 mmHg.\n- If the albumin to creatinine ratio is 70 mg/mmol or more: Below 130/80 mmHg.",more or \n• chronic kidne y disease plus albumin \nto creatinine ratio of 7 0 mg/mmol or \nmore Below \n130/80 NICE's guideline on type 1 \ndiabet es in adult s \n(recommendation 1 .13.8) \nNICE's guideline ...
1,"What challenges do healthcare providers face in diagnosing asthma in children, and what alternative methods does the committee suggest when standard diagnostic tests are unavailable or delayed?","Healthcare providers face challenges in diagnosing asthma in children due to difficulties in performing tests, as children may find some tests hard to perform and be unwilling to have blood tests. Additionally, access to standard diagnostic tests like spirometry and FeNO measurements can be delayed or unavailable.\n\nAs an alternative method, the committee suggests using peak expiratory flow (PEF) variability as a substitute rule-in test when standard tests are not available or there is a significant delay in obtaining them. They also noted that if there is diagnostic doubt after performing other tests, a referral to an asthma specialist for a second opinion and consideration of a bronchial challenge test should be made.","Healthcare providers face challenges in diagnosing asthma in children due to the difficulty some children have in performing certain tests and their unwillingness to undergo blood tests. The committee recognizes that standard diagnostic tests, such as bronchial challenge tests, are not easily available in many areas, which could delay diagnosis. As an alternative, when standard diagnostic tests are unavailable or delayed, the committee suggests managing children under 5 pragmatically based on symptoms and signs only. For older children, they recommend considering a combination of available tests that correlate, as no single test demonstrates high enough sensitivity and specificity to be diagnostic in all cases.","conditions#notice-of -right s).Page 45 of\n64 unlik ely and can be ruled out wit hout r esor ting t o bronchial challenge t esting. Alt hough \ntaking blood f or IgE is in vasive, it does ha ve the adv ..."
2,"What are the key research questions being addressed regarding the effectiveness of relaxation therapies and appropriate blood pressure targets for managing hypertension in adults with specific conditions, as outlined in the NICE recommendations?","The key research questions being addressed regarding the effectiveness of relaxation therapies and appropriate blood pressure targets for managing hypertension in adults include:\n\n1. **Relaxation Therapies**: What is the clinical and cost-effectiveness of relaxation therapies for managing primary hypertension in adults in terms of reducing cardiovascular events and improving quality of life?\n\n2. **Blood Pressure Targets**: \n - What are the optimal blood pressure targets in adults with hypertension and aortic aneurysm, and does this vary by age?\n - What are the optimal blood pressure targets in adults with prior ischaemic or haemorrhagic stroke?\n\nThese questions aim to help determine the best